#### Create Models (50 POINTS)
Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe. 

#### Model Advantages (10 POINTS)
Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.

#### Interpret Feature Importance (30 POINTS)
Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?

#### Interpret Support Vectors (10 POINTS)
Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model— then analyze the support vectors from the subsampled dataset.

# Mini-Lab

#### Alan Abadzic, John Girard, Eric Laigaie, Garrett Shankel

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("NY_Listings_Validated.csv")

### Create Models

#### Data Load-In and Preprocessing

In [65]:
# First, we need to create our classification column. Any rentals with a Review Scores Rating above 90 receive "A", else
# recieve "Not A".

def categorise(row):  
    if row['Review Scores Rating'] > 89:
        return 'A'
    else:
        return 'Not A'
    return 'IDK'

df['Grade'] = df.apply(lambda row: categorise(row), axis=1)

In [66]:
# Filter to only useful columns
data = df[['Host Response Rate', 'Host Is Superhost', 'Host total listings count', 'City', 'Room type',
          'Accommodates', 'Bathrooms', 'Bedrooms', 'Price', 'Minimum nights', 'Maximum nights', 'Availability 365',
          'Number of reviews', 'Reviews per month', 'Grade']]



# One-hot Encode
city_one_hot = pd.get_dummies(data['City'])
room_one_hot = pd.get_dummies(data['Room type'])

data = data.drop('City',axis = 1)
data = data.drop('Room type',axis = 1)

data = data.join(city_one_hot)
data = data.join(room_one_hot)


# Map boolean to integer
data["Host Is Superhost"] = data["Host Is Superhost"].astype(int)



# Scale Data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

grade = data['Grade']
to_scale = data.drop("Grade", axis = 1)
cols = to_scale.columns

scaled = scaler.fit_transform(to_scale)

data = pd.DataFrame(scaled, columns = cols)
data['Grade'] = grade

In [67]:
# Split into 80/20 train/test split.
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

print("Train: " + str(len(train)) + ", Test: " + str(len(test)))

train_y = train['Grade']
train_x = train.drop('Grade', axis=1)
test_y = train['Grade']
test_x = train.drop('Grade', axis=1)

Train: 35220, Test: 8805


#### Logistic Regression Testing

In [72]:
# Logistic Regression Model
from sklearn import metrics as mt
from sklearn.linear_model import LogisticRegression
import time

penalties = ['none', 'l1', 'l2', 'elasticnet']
print("Testing different penalties:")
for penalty in penalties:
    start = time.time()
    if penalty == 'elasticnet':
        clf = LogisticRegression(max_iter=1000, penalty=penalty, solver='saga', l1_ratio =0.5).fit(train_x, train_y)
    else:
        clf = LogisticRegression(max_iter=1000, penalty=penalty, solver='saga').fit(train_x, train_y)
    end = time.time()
    predictions = clf.predict(test_x)
    acc = round((mt.accuracy_score(predictions, test_y)) * 100,2)
    print(penalty + ": " + str(acc) + '%, Time: ' + str(end - start) + ' seconds')

print('\n')
Cs = [0.05, .5, 1, 100]
print("Testing different C values with Elastic Net penalty:")
for c in Cs:
    start = time.time()
    clf = LogisticRegression(max_iter=1000, penalty='elasticnet', solver='saga', l1_ratio=0.5, C=c).fit(train_x, train_y)
    end = time.time()
    predictions = clf.predict(test_x)
    acc = round((mt.accuracy_score(predictions, test_y)) * 100,2)
    print(str(c) + ": " + str(acc) + '%, Time: ' + str(end - start) + ' seconds')

Testing different penalties:
none: 71.67%, Time: 14.926925897598267 seconds
l1: 71.67%, Time: 0.6340394020080566 seconds
l2: 71.5%, Time: 0.5453207492828369 seconds
elasticnet: 71.64%, Time: 0.6053166389465332 seconds


Testing different C values with Elastic Net penalty:
0.05: 70.24%, Time: 0.38519811630249023 seconds
0.5: 71.56%, Time: 0.6578881740570068 seconds
1: 71.64%, Time: 0.6598906517028809 seconds
100: 71.67%, Time: 8.933096647262573 seconds


In [16]:
from sklearn.svm import SVC
import time

kernels = ['rbf', 'linear', 'poly', 'sigmoid']

print("Testing different kernels:")
for kernel in kernels:
    start = time.time()
    svm_clf = SVC(C=0.5, kernel=kernel, degree=3, gamma='auto').fit(train_x, train_y)
    end = time.time()
    predictions = svm_clf.predict(test_x)
    acc = str(round(mt.accuracy_score(predictions, test_y)*100, 2))
    print(kernel + ": " + acc + '%, Time: ' + str(end - start) + ' seconds')

Testing different kernels:
rbf: 63.09%, Time: 71.14248633384705 seconds
linear: 68.56%, Time: 45.71343541145325 seconds
poly: 62.93%, Time: 46.47894787788391 seconds
sigmoid: 63.21%, Time: 74.40014004707336 seconds


### Interpret Feature Importance

In [50]:
clf = LogisticRegression(max_iter=1000, penalty='elasticnet', solver='saga', l1_ratio=0.5, C=100).fit(train_x, train_y)
predictions = clf.predict(test_x)
acc = round((mt.accuracy_score(predictions, test_y)) * 100,2)

zip_vars = zip(clf.coef_.T,data.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', round(coef[0],4))

Reviews per month has weight of -14.027
Host Is Superhost has weight of -2.2139
Entire home/apt has weight of -0.504
Number of reviews has weight of -0.3621
Brooklyn has weight of -0.2467
Host Response Rate has weight of -0.1314
Availability 365 has weight of -0.0644
Manhattan has weight of -0.0513
Staten Island has weight of 0.0
Private room has weight of 0.0294
Queens has weight of 0.0733
Price has weight of 0.1288
Bronx has weight of 0.2149
Shared room has weight of 0.3109
Bathrooms has weight of 0.3293
Bedrooms has weight of 0.4319
Accommodates has weight of 1.1808
Minimum nights has weight of 1.5822
Host total listings count has weight of 2.5011
Maximum nights has weight of 5.8772


,Host Response Rate,Host Is Superhost,Host total listings count,Accommodates,Bathrooms,Bedrooms,Price,Minimum nights,Maximum nights,Availability 365,...,Reviews per month,Bronx,Brooklyn,Manhattan,Queens,Staten Island,Entire home/apt,Private room,Shared room,Grade
0,1.0,0.0,0.004086,0.000000,0.064516,0.000000,0.043043,0.000801,1.350418e-08,0.756164,...,0.072157,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,A
1,1.0,0.0,0.001021,0.000000,0.064516,0.000000,0.028028,0.000801,1.396984e-08,0.945205,...,0.062780,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Not A
2,1.0,0.0,0.016343,0.200000,0.193548,0.076923,0.080080,0.001601,1.396984e-08,0.972603,...,0.156135,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,A
3,0.7,0.0,0.001021,0.200000,0.064516,0.000000,0.140140,0.000801,5.234033e-07,0.980822,...,0.027313,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Not A
4,1.0,0.0,0.001021,0.066667,0.064516,0.000000,0.060060,0.000000,5.234033e-07,0.986301,...,0.150836,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44020,1.0,0.0,0.001021,0.066667,0.064516,0.000000,0.049049,0.000000,5.234033e-07,0.994521,...,0.050958,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,A
44021,1.0,0.0,0.001021,0.066667,0.064516,0.000000,0.055055,0.000801,1.862645e-09,0.000000,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,Not A
44022,1.0,0.0,0.001021,0.200000,0.064516,0.000000,0.050050,0.000000,2.793968e-09,0.980822,...,0.006930,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,Not A
44023,1.0,0.0,0.001021,0.200000,0.064516,0.000000,0.100100,0.001601,9.313226e-09,0.906849,...,0.007746,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Not A


### Interpret Support Vectors

In [64]:
#svm_clf = SVC(C=0.5, kernel='linear', degree=3, gamma='auto').fit(train_x, train_y)
#predictions = svm_clf.predict(test_x)

df_train = train_x
df_support = df_train.iloc[svm_clf.support_,:]

df_train['Grade'] = train_y
df_support['Grade'] = train_y[svm_clf.support_]

#df_support.info()

#df_tested_on = df_imputed.iloc[train_indices].copy() # saved from above, the indices chosen for training
# now get the support vectors from the trained model
#df_support = df_tested_on.iloc[svm_clf.support_,:].copy()

#df_support['Survived'] = y[svm_clf.support_] # add back in the 'Survived' Column to the pandas dataframe
#df_imputed['Survived'] = y # also add it back in for the original data
#df_support.info()

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([   17,    42,    60,    89,    91,\n            ...\n            35200, 35202, 35211, 35212, 35219],\n           dtype='int64', length=5307). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [21]:
# Grid search notes -- this takes a long time

#from sklearn.model_selection import GridSearchCV
#from sklearn.svm import SVC

#print('going')
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100], 'gamma':('scale', 'auto'), 'degree':[1,2,3]}
#svm_clf = SVC()
#clf = GridSearchCV(svm_clf, parameters)
#clf.fit(train_x, train_y)